In [5]:
import geopandas
import pandas
import pyodbc
import numpy as np


In [6]:
server = "213.140.22.237\SQLEXPRESS"
database = "zanella.luca"
username = "zanella.luca"
password = "xxx123##"

In [7]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [8]:
query_somm_vacc = 'SELECT * FROM dbo.puntiSomministrazioneVaccini'
somm_vacc = pandas.read_sql_query(query_somm_vacc,conn)

In [9]:
somm_vacc

,column1,area,codice_NUTS1,codice_NUTS2,codice_regione_ISTAT,denominazione_struttura,lat,lng,nome_area,tipologia
0,0,ABR,ITF,ITF1,13,AVEZZANO-SULMONA-L'AQUILA,42.051257,13.934312,Abruzzo,Ospedaliero
1,1,ABR,ITF,ITF1,13,CASA DI CURA VILLA SERENA,42.521354,14.097955,Abruzzo,Ospedaliero
2,2,ABR,ITF,ITF1,13,CDC S.CAMILLA-C.RIA.VILLA PINI,42.379116,14.189817,Abruzzo,Territoriale
3,3,ABR,ITF,ITF1,13,CENTRO RIABILITAZIONE SANEX,42.712143,13.657788,Abruzzo,Territoriale
4,4,ABR,ITF,ITF1,13,CONS. S.STEFANO S.EGIDIO,42.823682,13.716292,Abruzzo,Territoriale
...,...,...,...,...,...,...,...,...,...,...
1988,1988,VEN,ITH,ITH3,5,PSR MOBY DICK,40.781324,-73.973988,Veneto,Territoriale
1989,1989,VEN,ITH,ITH3,5,PSR NO PROBLEM COOP. STUDIO PROGETTO,45.441466,12.315259,Veneto,Territoriale
1990,1990,VEN,ITH,ITH3,5,RSA NOI CON VOI - MOGLIANO VENETO,45.656641,12.241009,Veneto,Territoriale
1991,1991,LOM,ITC,ITC4,3,PRESIDIO TERRITORIALE IPPOCRATE,45.521342,9.163329,Lombarida,Territoriale


In [10]:
from shapely.geometry import Point

punto_utente = Point([45.4978672,9.1746517][::-1])
punto = geopandas.GeoSeries([punto_utente],crs="EPSG:4326").to_crs(epsg=3857)
buffer = punto.buffer(5000)
somm_vacc = geopandas.GeoDataFrame(somm_vacc,geometry=geopandas.points_from_xy(somm_vacc['lng'],somm_vacc['lat']))
somm_vacc.crs = "EPSG:4326"

In [11]:
somm_vacc = somm_vacc.to_crs(epsg=3857)
buffer = buffer.to_crs(epsg=3857)

In [12]:
vacc = somm_vacc[somm_vacc.within(buffer.geometry.squeeze())]
vacc

,column1,area,codice_NUTS1,codice_NUTS2,codice_regione_ISTAT,denominazione_struttura,lat,lng,nome_area,tipologia,geometry
669,669,LOM,ITC,ITC4,3,ASST GRANDE OSPEDALE METROPOLITANO NIGUARDA,45.510049,9.186536,Lombardia,Ospedaliero,POINT (1022640.465 5702178.888)
672,672,LOM,ITC,ITC4,3,ASST NORD MILANO,45.466574,9.171866,Lombardia,Ospedaliero,POINT (1021007.497 5695275.570)
705,705,LOM,ITC,ITC4,3,CASA DI CURA S. PIO X - MILANO,45.494766,9.189350,Lombardia,Ospedaliero,POINT (1022953.818 5699751.499)
706,706,LOM,ITC,ITC4,3,CASA DI CURA S.CAMILLO,45.481789,9.204472,Lombardia,Territoriale,POINT (1024637.181 5697690.853)
735,735,LOM,ITC,ITC4,3,IST.AUXOLOGICO ITAL.-I.S. S.MICHELE-MILA,45.469474,9.165508,Lombardia,Ospedaliero,POINT (1020299.639 5695735.923)
738,738,LOM,ITC,ITC4,3,IST.CLINICO S.AMBROGIO SPA-MILANO,45.484423,9.153459,Lombardia,Ospedaliero,POINT (1018958.395 5698109.111)
750,750,LOM,ITC,ITC4,3,ISTITUTO ORTOPEDICO GALEAZZI SPA - MILAN,45.523162,9.174005,Lombardia,Ospedaliero,POINT (1021245.565 5704262.114)
1991,1991,LOM,ITC,ITC4,3,PRESIDIO TERRITORIALE IPPOCRATE,45.521342,9.163329,Lombarida,Territoriale,POINT (1020057.174 5703972.960)
1992,1992,LOM,ITC,ITC4,3,OSPEDALE FIERA MILANO,45.483415,9.149910,Lombardia,Ospedaliero,POINT (1018563.311 5697949.097)


In [13]:
coordinate = np.array(vacc[['lat','lng','denominazione_struttura']])

In [14]:
result = ""
for cord in coordinate:
    result += "[" + str(cord[1]) + "," + str(cord[0]) + ","  + "'"  + str   (cord[2]) + "'" + "],"

#la lunghezza di result - l'ultimo carattere che è la virgola che non mi serve più le quadre è per l'array muldidimansionale
result = "[" + result[0:len(result) -1] + "]"

In [15]:
result

"[[9.1865356,45.5100491,'ASST GRANDE OSPEDALE METROPOLITANO NIGUARDA'],[9.171866399999999,45.4665742,'ASST NORD MILANO'],[9.1893505,45.494766,'CASA DI CURA S. PIO X - MILANO'],[9.2044724,45.4817887,'CASA DI CURA S.CAMILLO'],[9.1655076,45.4694744,'IST.AUXOLOGICO ITAL.-I.S. S.MICHELE-MILA'],[9.153459000000002,45.484423,'IST.CLINICO S.AMBROGIO SPA-MILANO'],[9.174005,45.523162,'ISTITUTO ORTOPEDICO GALEAZZI SPA - MILAN'],[9.1633295,45.5213421,'PRESIDIO TERRITORIALE IPPOCRATE'],[9.1499099,45.4834152,'OSPEDALE FIERA MILANO']]"

In [16]:
import pymssql

server = "213.140.22.237\SQLEXPRESS"
database = "zanella.luca"
username = "zanella.luca"
password = "xxx123##"

conn_pymssql = pymssql.connect(server,username,password,database)
cursor = conn_pymssql.cursor()

In [17]:
query_somm_vacc = 'SELECT * FROM dbo.puntiSomministrazioneVaccini'
somm_vacc = pandas.read_sql_query(query_somm_vacc,conn_pymssql)
somm_vacc

,column1,area,codice_NUTS1,codice_NUTS2,codice_regione_ISTAT,denominazione_struttura,lat,lng,nome_area,tipologia
0,0,ABR,ITF,ITF1,13,AVEZZANO-SULMONA-L'AQUILA,42.051257,13.934312,Abruzzo,Ospedaliero
1,1,ABR,ITF,ITF1,13,CASA DI CURA VILLA SERENA,42.521354,14.097955,Abruzzo,Ospedaliero
2,2,ABR,ITF,ITF1,13,CDC S.CAMILLA-C.RIA.VILLA PINI,42.379116,14.189817,Abruzzo,Territoriale
3,3,ABR,ITF,ITF1,13,CENTRO RIABILITAZIONE SANEX,42.712143,13.657788,Abruzzo,Territoriale
4,4,ABR,ITF,ITF1,13,CONS. S.STEFANO S.EGIDIO,42.823682,13.716292,Abruzzo,Territoriale
...,...,...,...,...,...,...,...,...,...,...
1988,1988,VEN,ITH,ITH3,5,PSR MOBY DICK,40.781324,-73.973988,Veneto,Territoriale
1989,1989,VEN,ITH,ITH3,5,PSR NO PROBLEM COOP. STUDIO PROGETTO,45.441466,12.315259,Veneto,Territoriale
1990,1990,VEN,ITH,ITH3,5,RSA NOI CON VOI - MOGLIANO VENETO,45.656641,12.241009,Veneto,Territoriale
1991,1991,LOM,ITC,ITC4,3,PRESIDIO TERRITORIALE IPPOCRATE,45.521342,9.163329,Lombarida,Territoriale
